# ALIGNN/ALIGNN-d graph representation for molecular structures

This notebook will guide you through encoding molecular structures into the ALIGNN/ALIGNN-d graph representation.

You will need to install `ase` for reading molecular structure files and creating molecular graphs.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import ase.io

## Read structure files

In [2]:
d_mol = ase.io.read('./data/D-alanine.mol')
l_mol = ase.io.read('./data/L-alanine.mol')

# Remove hydrogens
d_mol = d_mol[d_mol.numbers != 1]
l_mol = l_mol[l_mol.numbers != 1]

## Convert to PyG graph data

In the ALIGNN formulation, the input representation consists of two graphs: an original atomic graph ("G") and its angular graph ("A"). The cell block below uses helper functions (which have dependency on `ase`) from `graphite` to extract atom type, bond lengths, bond angles, and dihedral angles and store them in a custom PyG `Data` object with the following fields:
- `edge_index_G`: atomic graph ("G") connectivity
- `edge_index_A`: angular graph ("A") connectivity
- `x_atm`: atom type information (in onehot encoding)
- `x_bnd`: bond lengths
- `x_ang`: bond angles and optionally dihedral angles (in radians)
- `mask_dih_ang`: mask vector indicating which angles are dihedral angles, if any

To omit dihedral angles (i.e., the original ALIGNN formulation), skip the dihedral angle calculation and set `mask_dih_ang` to `None`.

In [3]:
from ase.neighborlist import neighbor_list
from sklearn.preprocessing import OneHotEncoder
from graphite.utils.alignn import line_graph, dihedral_graph, get_bnd_angs, get_dih_angs
from graphite.data import AngularGraphPairData

# OVITO's default element-specific cutoffs for creating molecular bonds
ovito_cutoff = {
    ('H', 'C'): 1.74, ('H', 'N'): 1.65,  ('H', 'O'): 1.632,
    ('C', 'C'): 2.04, ('C', 'N'): 1.95,  ('C', 'O'): 1.932,
    ('N', 'N'): 1.86, ('N', 'O'): 1.842, ('O', 'O'): 1.824,
}

def get_molecular_graph(atoms):
    """Returns edge indices of strong chemical bonds according to a pre-defined
    element-specific cutoff criteria.  
    """
    dummy_cell = np.diag([30, 30, 30])
    dummy_pbc  = np.array([False]*3)
    i, j, d = neighbor_list('ijd', atoms, cutoff=ovito_cutoff)
    return np.stack((i, j)), d

def atoms2pygdata(atoms):
    """Converts ASE `atoms` into a PyG graph data holding the molecular graph (G) and the angular graph (A).
    The angular graph holds both bond and dihedral angles.
    """
    edge_index_G, x_bnd = get_molecular_graph(atoms)
    edge_index_bnd_ang = line_graph(edge_index_G)
    edge_index_dih_ang = dihedral_graph(edge_index_G)
    edge_index_A = np.hstack([edge_index_bnd_ang, edge_index_dih_ang])
    x_atm = OneHotEncoder(sparse_output=False).fit_transform(atoms.numbers.reshape(-1,1))
    x_bnd_ang = get_bnd_angs(atoms, edge_index_G, edge_index_bnd_ang)
    x_dih_ang = get_dih_angs(atoms, edge_index_G, edge_index_dih_ang)
    x_ang = np.concatenate([x_bnd_ang, x_dih_ang])
    mask_dih_ang = [False]*len(x_bnd_ang) + [True]*len(x_dih_ang)
    
    data = AngularGraphPairData(
        edge_index_G = torch.tensor(edge_index_G, dtype=torch.long),
        edge_index_A = torch.tensor(edge_index_A, dtype=torch.long),
        x_atm        = torch.tensor(x_atm,        dtype=torch.float),
        x_bnd        = torch.tensor(x_bnd,        dtype=torch.float),
        x_ang        = torch.tensor(x_ang,        dtype=torch.float),
        mask_dih_ang = torch.tensor(mask_dih_ang, dtype=torch.bool),
    )
    return data

Using toy data, L-alanine and D-alanine, as examples, we can see that while all the bond angles are the same between the L- and D- counterparts, some of the dihedral angles are clearly different. Many conventional graph neural networks cannot capture such chiral distinctions.

By the way, the angles are double-counted because the atomic graph (and its angular graph) is bidirectional under the GNN/message-passing formulation.

In [4]:
import pandas as pd

l_mol_data = atoms2pygdata(l_mol)
d_mol_data = atoms2pygdata(d_mol)

angles = pd.DataFrame(
    {
        'Is a dihedral angle': l_mol_data.mask_dih_ang.numpy(),
        'l-mol angles': l_mol_data.x_ang.rad2deg(),
        'd-mol angles': d_mol_data.x_ang.rad2deg(),
    }
)
pd.set_option("display.precision", 1)
angles

,Is a dihedral angle,l-mol angles,d-mol angles
0,False,109.7,109.7
1,False,108.1,108.1
2,False,109.7,109.7
3,False,110.8,110.8
4,False,108.1,108.1
5,False,110.8,110.8
6,False,123.2,123.2
7,False,110.4,110.4
8,False,123.2,123.2
9,False,126.4,126.4


## Define model architecture

Following DeepMind's nomenclature in the MeshGraphNets formulation, a graph (message-passing) neural network can be described as three components:
- Encoder
    - Initial encoding/embedding that transforms graph-based data into whatever format neccessary for graph convolutions or message-passing.
    - Often customized depending on the user's data format. For example, the original ALIGNN work also encodes bond types (e.g., single-bond, double-bond, triple-bond, etc.) but such information is not encoded in this example.
- Processor
    - Graph convolutions or message-passing layers. Refer to the ALIGNN paper for how it is implemented.
    - Normally does not need to be customized.
- Decoder
    - Final operation that transforms latent node/edge features into an appropriate output format.
    - Often customized by the user for a specific output format. For example, local nodal output vs. global pooled output.

In [5]:
from graphite.nn.models.alignn import Encoder, Processor, Decoder, ALIGNN

# In case it has not been made clear, this ALIGNN implementaition can encode dihedral angles
gnn = ALIGNN(
    encoder   = Encoder(num_species=3, cutoff=2.0, dim=128, dihedral=True),
    processor = Processor(num_convs=5, dim=128),
    decoder   = Decoder(node_dim=128, out_dim=2),
)

## Model forward pass

For now I'm only demonstrating a successful forward pass. Model training is currently omitted.

In [6]:
gnn(l_mol_data.clone())

tensor([[ 0.1284,  0.4240],
        [ 0.1206,  0.3751],
        [-0.0239,  0.5594],
        [ 0.2630,  0.2095],
        [-0.0830,  0.4637],
        [ 0.0106,  0.2681]], grad_fn=<AddmmBackward0>)

## Interpretation analysis

Compared to other models, ALIGNN is well suited for interpretation analysis, as the atoms, bonds, bond angles, and optionally dihedral angles are all explicitly represented as graph components. Therefore, for a given prediction, one can examine the attribution/sensitivity of each atom/bond/angle component.

Here I provide two ways to approach ALIGNN interpretation:
- gradient analysis
- summation of positive scalars

### Gradient analysis

This is a rather common and simple approach for interpreting a neural network. For a given (high-dimensional) input $\mathbf{x}$ and its corresponding (scalar) output $y$, we can compute the gradient $\frac{\delta y}{\delta \mathbf{x}}$, which tells us the attribution/sensitivity of the individual elements in $\mathbf{x}$ to the output $y$.

Let's first write a custom Decoder that pools the latent features of all components into a global scalar quantity.

In [7]:
from torch import nn
from torch_geometric.utils import scatter
from graphite.nn import MLP

class GlobalScalarDecoder(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.decoder = MLP([dim, dim, 1], act=nn.SiLU())

    def forward(self, data):
        # Globally pool all atom/bond/angle components into a single feature vector
        # This operation assumes all components have the same dimensionality, 
        # which should be the case for ALIGNN.
        if data.batch is not None:
            h_atm_pooled = scatter(data.h_atm, data.x_atm_batch, dim=0, reduce='mean')
            h_bnd_pooled = scatter(data.h_bnd, data.x_bnd_batch, dim=0, reduce='mean')
            h_ang_pooled = scatter(data.h_ang, data.x_ang_batch, dim=0, reduce='mean')
            h_pooled = (h_atm_pooled + h_bnd_pooled + h_ang_pooled) / 3
        else:
            h_pooled = (data.h_atm.mean(dim=0) + data.h_bnd.mean(dim=0) + data.h_ang.mean(dim=0)) / 3
        return self.decoder(h_pooled)

How the pooling is performed, e.g, via summing or averaging, can be important. If the global property is instrinsic, i.e., does not depend on system size, then averaging is preferred over summing.

On a side note, when batching multiple molecular graphs into a single `data` object, remember to keep track of how `x_atm`, `x_bnd`, and `x_ang` are batched via the use of the `follow_batch` argument. Here is an example:
```python
from torch_geometric.loader import DataLoader

loader = DataLoader([l_mol_data, d_mol_data], batch_size=2, follow_batch=['x_atm', 'x_bnd', 'x_ang'])
batch = next(iter(loader))
output = gnn(batch)
```

The code below shows that indeed a scalar output is given for a molecular graph input.

In [8]:
gnn = ALIGNN(
    encoder   = Encoder(num_species=3, cutoff=2.0, dim=128, dihedral=True),
    processor = Processor(num_convs=5, dim=128),
    decoder   = GlobalScalarDecoder(dim=128),
)

gnn(l_mol_data.clone())

tensor([-0.3686], grad_fn=<AddBackward0>)

We will now adjust the code above to compute the gradient. However, note that computing the gradient  $\frac{\delta y}{\delta \mathbf{x}}$ requires $\mathbf{x}$ to be a continuous, smooth variable. Unfortunately, the atom type is a discrete variable, so we cannot take gradient w.r.t. atom types (technically, we could, but the value probably would not make sense).

In [9]:
l_mol_data.x_bnd.requires_grad_(True)
l_mol_data.x_ang.requires_grad_(True)

y = gnn(l_mol_data)
grad_bnd, grad_ang = torch.autograd.grad(y, [l_mol_data.x_bnd, l_mol_data.x_ang])
print(grad_bnd)
print(grad_ang)

tensor([ 2.1229,  3.5029,  3.7278,  0.9814, -1.5923,  0.2125, -2.0478, -1.1346,
         1.6339, -0.0160])
tensor([-0.0009,  0.0198,  0.0020, -0.0182,  0.0287, -0.0141,  0.0055, -0.0202,
        -0.0019,  0.0154, -0.0037,  0.0094, -0.0069, -0.0070,  0.0027,  0.0079,
        -0.0066, -0.0007, -0.0065,  0.0124])


### Summation of positive scalars

In the case that the scalar output is stricly positive, then one may choose to transform all components to hold positive scalars, and then sum into a single scalar. This way, the individual scalars at the atom/bond/angle components may be interpreted as contributions towards the global output.

Here is the corresponding custom Decoder.

In [10]:
from torch import nn
from torch_geometric.utils import scatter
from graphite.nn import MLP

class PositiveScalarsDecoder(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        self.transform_atm = nn.Sequential(MLP([dim, dim, 1], act=nn.SiLU()), nn.Softplus())
        self.transform_bnd = nn.Sequential(MLP([dim, dim, 1], act=nn.SiLU()), nn.Softplus())
        self.transform_ang = nn.Sequential(MLP([dim, dim, 1], act=nn.SiLU()), nn.Softplus())

    # Here I assume `data` is not batched.
    def forward(self, data):
        atm_scalars = self.transform_atm(data.h_atm)
        bnd_scalars = self.transform_bnd(data.h_bnd)
        ang_scalars = self.transform_bnd(data.h_ang)
        return (atm_scalars, bnd_scalars, ang_scalars)

The code below shows that the outputs are all positive scalars for all of the atoms, bonds and angles.

In [11]:
gnn = ALIGNN(
    encoder   = Encoder(num_species=3, cutoff=2.0, dim=128, dihedral=True),
    processor = Processor(num_convs=5, dim=128),
    decoder   = PositiveScalarsDecoder(dim=128),
)

gnn(l_mol_data.clone())

(tensor([[0.4597],
         [0.4562],
         [0.4074],
         [0.2855],
         [0.4020],
         [0.3290]], grad_fn=<SoftplusBackward0>),
 tensor([[0.5128],
         [0.5722],
         [0.8208],
         [0.5931],
         [1.1574],
         [0.8752],
         [0.9224],
         [0.7308],
         [0.6113],
         [0.7641]], grad_fn=<SoftplusBackward0>),
 tensor([[0.7636],
         [0.7443],
         [0.6235],
         [0.8164],
         [0.6972],
         [0.7830],
         [0.7203],
         [0.7038],
         [0.6038],
         [0.8213],
         [0.5998],
         [0.7990],
         [0.6479],
         [0.7133],
         [0.6365],
         [0.7737],
         [0.5933],
         [0.7773],
         [0.5984],
         [0.8082]], grad_fn=<SoftplusBackward0>))